In [ ]:
import os
from crewai import Agent, Task, Crew
from crewai.process import Process


# 1. Initialize the Language Model (LLM)
# It is assumed that the OPENAI_API_KEY environment variable is set.
# ChatOpenAI uses the environment variable automatically.
# You can replace this with any other supported LLM by CrewAI/LangChain,
# such as Groq, Mistral, Ollama, etc.

llm = ChatOpenAI(model="gpt-4o-mini")

# 2. Define the Agent(s)

researcher = Agent(
    role='Senior Research Analyst',
    goal='Discover and summarize the latest trends in renewable energy',
    backstory='An expert in sustainability and market analysis. Meticulous and detail-oriented.',
    verbose=True,
    allow_delegation=False,
    llm=llm  # Inject the initialized LLM here
)

# 3. Define the Task(s)

research_task = Task(
    description=(
        "Analyze the current global news and market data for renewable energy. "
        "Specifically focus on **solar panel efficiency** improvements and **battery storage** "
        "innovations over the last six months. "
        "The final output must be a concise, three-paragraph summary."
    ),
    expected_output='A three-paragraph summary report on solar and battery advancements.',
    agent=researcher  # Assign the task to the defined agent
)

# 4. Create the Crew and Run the Process

# For simplicity, this example uses a single agent and a single task.
# A full crew typically has multiple agents and multiple tasks.
project_crew = Crew(
    agents=[researcher],
    tasks=[research_task],
    process=Process.sequential,  # Execute tasks in order
    verbose=2  # High verbosity for observing the agent's thought process
)

print("### Crew Starting Execution ###")

# Prompting the LLM is implicitly handled when the crew is kicked off,
# as the agents use the LLM to process their assigned tasks and goals.
result = project_crew.kickoff()

# 5. Output the Result

print("\n\n### Task Execution Complete ###")
print(f"Final Report:\n{result}")

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
"""
Extract the claims and return a JSON for each claim retrieved from the argument.
"""

In [ ]:
import os
from crewai import Agent, Task, Crew, LLM
from crewai.process import Process

# Ensure the environment variable is set before running:
# export GEMINI_API_KEY="YOUR_API_KEY"

# 1. Initialize the LLM for Gemini
# CrewAI uses a simplified LLM class that interfaces with LiteLLM.
# The model name must be provided in the format: <provider>/<model_name>
# Common Gemini models: gemini/gemini-2.5-flash, gemini/gemini-2.5-pro

from dotenv import load_dotenv

load_dotenv()

gemini_llm = LLM(
    model='gemini/gemini-2.5-pro',
    # LiteLLM automatically uses GEMINI_API_KEY from environment
    temperature=0.1
)

# 2. Define the Agent, injecting the Gemini LLM
gemini_researcher = Agent(
    role='AI Configuration Specialist',
    goal='Provide a concise, detailed report on Gemini LLM integration into CrewAI.',
    backstory='An expert in multi-agent orchestration frameworks and Google AI APIs. Highly accurate and focused on syntax.',
    verbose=True,
    allow_delegation=False,
    llm=gemini_llm # <-- Inject the configured Gemini LLM here
)

# 3. Define the Task
integration_task = Task(
    description=(
        "Summarize the Python steps required to initialize a CrewAI Agent using the 'gemini/gemini-2.5-flash' model. "
        "The summary must explicitly mention the required environment variable for the API key."
    ),
    expected_output='A four-step guide covering installation, API key variable, LLM initialization syntax, and Agent injection.',
    agent=gemini_researcher
)

# 4. Create and Run the Crew
project_crew = Crew(
    agents=[gemini_researcher],
    tasks=[integration_task],
    process=Process.sequential,
    verbose=1
)

print("### Crew Starting Execution with Gemini ###")

result = project_crew.kickoff()

print("\n\n### Gemini Crew Task Complete ###")
print(f"Final Report:\n{result}")

### Crew Starting Execution with Gemini ###


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 9f4ea368-b2b2-452b-bcd2-7c61dfad7c9d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: AI Configuration Specialist                                                                             │
│                                                                                                                 │
│  Task: Summarize the Python steps required to initialize a CrewAI Agent using the 'gemini/gemini-2.5-flash'     │
│  model. The summary must explicitly mention the required environment variable for the API key.                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/home/rg/Documents/GitHub/MumbaiHacks/.venv/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: AI Configuration Specialist                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### Report: Integrating Gemini 1.5 Flash with CrewAI                                                           │
│                                                                                                                 │
│  This guide provides the precise Python steps to configure and initialize a CrewAI Agent utilizing Google's     │
│  `gemini-1.5-flash-latest` model. The process is broken down into four essential stages: dependency             │
│  installation, API key configuration, LLM object initialization, and injection into the CrewAI Agent.           │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  #### **Step 1: Install Required Libraries**                                                                    │
│                                                                                                                 │
│  First, ensure your Python environment has the necessary packages. You will need `crewai`, `crewai-tools`, and  │
│  the specific Google Generative AI library that provides the LangChain integration used by CrewAI.              │
│                                                                                                                 │
│  Execute the following command in your terminal to install all required dependencies:                           │
│                                                                                                                 │
│  ```bash                                                                                                        │
│  pip install crewai crewai-tools langchain-google-genai                                                         │
│  ```                                                                                                            │
│                                                                                                                 │
│  *   `crewai`: The core multi-agent orchestration framework.                                                    │
│  *   `crewai-tools`: Provides common tools and utilities for agents.                                            │
│  *   `langchain-google-genai`: The specific integration library required to interface with Google's Gemini      │
│  models via LangChain, which CrewAI uses under the hood.                                                        │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  #### **Step 2: Configure the Google API Key Environment Variable**                                             │
│                                                                                                                 │
│  The Google Generative AI SDK requires an API key for authentication. The standard and required environment     │
│  variable for this key is `GOOGLE_API_KEY`.            



### Gemini Crew Task Complete ###
Final Report:
### Report: Integrating Gemini 1.5 Flash with CrewAI

This guide provides the precise Python steps to configure and initialize a CrewAI Agent utilizing Google's `gemini-1.5-flash-latest` model. The process is broken down into four essential stages: dependency installation, API key configuration, LLM object initialization, and injection into the CrewAI Agent.

---

#### **Step 1: Install Required Libraries**

First, ensure your Python environment has the necessary packages. You will need `crewai`, `crewai-tools`, and the specific Google Generative AI library that provides the LangChain integration used by CrewAI.

Execute the following command in your terminal to install all required dependencies:

```bash
pip install crewai crewai-tools langchain-google-genai
```

*   `crewai`: The core multi-agent orchestration framework.
*   `crewai-tools`: Provides common tools and utilities for agents.
*   `langchain-google-genai`: The specific inte

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 5593f98a-ea48-435b-b8a9-241e58287170                                                                     │
│  Agent: AI Configuration Specialist                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 9f4ea368-b2b2-452b-bcd2-7c61dfad7c9d                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ### Report: Integrating Gemini 1.5 Flash with CrewAI                                             │
│                                                                                                                 │
│  This guide provides the precise Python steps to configure and initialize a CrewAI Agent utilizing Google's     │
│  `gemini-1.5-flash-latest` model. The process is broken down into four essential stages: dependency             │
│  installation, API key configuration, LLM object initialization, and injection into the CrewAI Agent.           │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  #### **Step 1: Install Required Libraries**                                                                    │
│                                                                                                                 │
│  First, ensure your Python environment has the necessary packages. You will need `crewai`, `crewai-tools`, and  │
│  the specific Google Generative AI library that provides the LangChain integration used by CrewAI.              │
│                                                                                                                 │
│  Execute the following command in your terminal to install all required dependencies:                           │
│                                                                                                                 │
│  ```bash                                                                                                        │
│  pip install crewai crewai-tools langchain-google-genai                                                         │
│  ```                                                                                                            │
│                                                                                                                 │
│  *   `crewai`: The core multi-agent orchestration framework.                                                    │
│  *   `crewai-tools`: Provides common tools and utilities for agents.                                            │
│  *   `langchain-google-genai`: The specific integration library required to interface with Google's Gemini      │
│  models via LangChain, which CrewAI uses under the hood.                                                        │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  #### **Step 2: Configure the Google API Key Environment Variable**                                             │
│                                                                                                                 │
│  The Google Generative AI SDK requires an API key for 

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



╭────────────────────────────── Execution Traces ──────────────────────────────╮
│                                                                              │
│  🔍 Detailed execution traces are available!                                 │
│                                                                              │
│  View insights including:                                                    │
│    • Agent decision-making process                                           │
│    • Task execution flow and timing                                          │
│    • Tool usage details                                                      │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯
Would you like to view your execution traces? [y/N] (20s timeout): 

╭────────────────────────── Tracing Preference Saved ──────────────────────────╮
│                                      